In [ ]:
from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config
from phdconf.config import * 

import os

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)

In [ ]:
# make crim queries different colour in plots 
s = {43, 44, 45, 46, 114}
for k, v in queries.items():
    v['type'] = 'specific' if k in s else 'broad'
#     if k == 44 or k == 93:
#         v['type'] = 'general'
    queries[k] = v

In [ ]:
index_names = ['filtered-phrasestop']#, config.SIGIR_INDEX_NAME]
qrel_paths = [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['aus']#, 'SIGIR']

mu = 1050

base_df = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1)[0][0]
base_query_df = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1, per_query=True)[0][0]

In [ ]:
runs = ['case-topics-crim-cases-{0}.run'.format(x) for x in ['in', 'out', 'all']]
run_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, "boosted"), runs, per_query=True)

In [ ]:
# excluding crim cases 
# plot_diff(run_dfs[1], base_query_df[0][0], config.METRIC_NAMES, queries)

In [ ]:
def get_diff(a, b, metric):
    diff = a-b
    print(diff[diff[metric] != 0.0][metric].sort_values())

# get_diff(run_dfs[1], base_query_df[0][0], 'rbp@0.80')

In [ ]:
def get_num_diff(a, b, metrics):
    diff = a-b
    diff = diff.astype(bool).sum(axis=0)/len(diff)
    print(diff[diff.index.isin(metrics)].rename(metrics).round(4).to_latex())
    
# get_num_diff(run_dfs[0], base_query_df[0][0], config.METRIC_NAMES)

In [ ]:
import copy

metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']

exclude_crim = [43, 44, 45, 46, 59, 64, 71, 104, 114, 117]
exclude_costs = [49, 64, 77, 78, 85, 86, 87, 116]

for r, ex in zip(['crim', 'costs'], [exclude_crim, exclude_costs]):
    print(r, ex)
    runs = ['case-topics-{0}-cases-{1}.run'.format(r, x) for x in ['in', 'out', 'all']]
    run_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, "boosted"), runs, per_query=True)
    b = base_query_df.iloc[~(base_query_df.index.isin(ex))]
    a = run_dfs[1].iloc[~(run_dfs[1].index.isin(ex))]
    diff = a-b
    for m in metrics:
        print(stats.ttest_rel(a[m].values, b[m].values).pvalue)
    print(diff[metrics].rename(metrics, axis='columns').mean())
    fig = diff[metrics].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')
    # fig.get_figure().savefig('figures/ausnl-{0}-filtered.pdf'.format(r))

In [ ]:
for m in metrics:
    print(b[b[m].isnull()][m])

In [ ]:
def get_diff_av_exclude(a, b, metrics, exclude):
    diff = a-b
    diff = diff[metrics]
    diff = diff.iloc[~(diff.index.isin(exclude))]
    diff = diff.mean()
    for m in metrics:
        print(diff[m])
        p = stats.ttest_rel(a[m], b[m]).pvalue
        if p < 0.01:
            diff[m] = "{:.4f}".format(diff[m]) + "$^{**}$"
        elif p < 0.05:
            diff[m] = "{:.4f}".format(diff[m]) + "$^{*}$"
        else:
            diff[m] = "{:.4f}".format(diff[m])
    
    return diff.rename(metrics)
    
# excluded_diff = get_diff_av_exclude(run_dfs[1], base_query_df, config.METRIC_NAMES, exclude_crim)

metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
ex_df = pd.DataFrame()
for r, ex in zip(['crim', 'costs'], [exclude_crim, exclude_costs]):
    runs = ['case-topics-{0}-cases-{1}.run'.format(r, x) for x in ['out']]
    run_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, "boosted"), runs, per_query=True)
    diff = get_diff_av_exclude(run_dfs[0], base_query_df, metrics, ex)
    ex_df[r] = diff

In [ ]:
# print(excluded_diff.round(4).to_latex())
# write_table('tables/filter-type', bold_max(ex_df).T.to_latex(escape=False))

In [ ]:
def read_crim_file(path: str):
    lookup = set()
    with open(path) as f:
        for line in f:
            parts = line.split()
            if parts[1] == '1':
                lookup.add(parts[0])
    return lookup

def count_crim_in_res_file(path: str, crim_lookup):
    qry = {}
    with open(path) as f:
        for line in f: 
            parts = line.split()
            v = qry.get(parts[0], 0)
            if parts[2] in crim_lookup: 
                v += 1
            qry[parts[0]] = v
                           
    return qry

crim_lookup = read_crim_file('../features/crim-cases.txt')
costs_lookup = read_crim_file('../features/costs-cases.txt')
crim_counts = count_crim_in_res_file(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-stop-unigram_dir_mu_1050.00.run', crim_lookup)

In [ ]:
def find_lookup_rel(path: str, lookup):
    potential = {}
    with open(path) as f:
        for line in f:
            parts = line.split()
            if parts[2] in lookup and parts[3] != '0':
                if parts[0] not in potential: 
                    potential[parts[0]] = [[], []]
                potential[parts[0]][0].append(parts[2])
                potential[parts[0]][1].append(parts[3])
    return potential
                
rel_crim = find_lookup_rel(config.AUS_QREL_PATH, crim_lookup)
rel_costs = find_lookup_rel(config.AUS_QREL_PATH, costs_lookup)

In [ ]:
for q, rel_docs in rel_crim.items(): 
    print(q, queries.get(int(q)), len(rel_docs[0]))


In [ ]:
for q, rel_docs in rel_costs.items(): 
    print(q, queries.get(int(q)), len(rel_docs[0]))

In [ ]:
def create_crim_count_plot(counts):
    fig = plt.figure() 
    ax = fig.add_subplot(111)
    fig.set_size_inches(16, 10)
    ticks = [str(x[0]) for x in sorted(crim_counts.items(), key=lambda x: x[1], reverse=True)]
    count_vals = sorted(crim_counts.values(), reverse=True)
    x = [i for i in range(len(counts))]
    ax.bar(x, count_vals)
    ax.set_xticks(np.arange(len(x)))
    ax.set_xticklabels(ticks, rotation=-90)
    return fig

count_plt = create_crim_count_plot(crim_counts)
# count_plt.savefig('figures/crim-counts.pdf')